<a href="https://colab.research.google.com/github/sljm12/machine_learning_notebooks/blob/master/LSTM_Accerlormeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf
from tensorflow.keras.layers  import Layer, LSTM, Dense
from tensorflow.keras.models import Sequential

import pandas as pd
from numpy import array
from sklearn.model_selection import train_test_split
import numpy as np

import glob,os

In [3]:
tf.__version__

'2.6.0'

In [4]:
nfeatures = 3 # the features are X,Y,Z
window_size = 3 # the number of records of X,Y,Z that we will count as one time sequence

## Test Data loading functions

In [31]:
walking = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/jumping-1.csv", delimiter=",",header=None, names=["X","Y","Z"])

In [32]:
walking.head()

,X,Y,Z
0,4.11686,1.58412,7.59052
1,3.72310,1.70451,7.80389
2,2.97199,1.54113,9.03701
3,2.63141,1.47678,9.83689
4,2.80433,1.42728,9.98384


In [4]:
len(walking)

758

In [ ]:
before1 = walking.loc[0]
before1["X"]

-1056

In [5]:
def convert_to_array(df, loc):
  row = df.loc[loc]
  return [row["X"],row["Y"], row["Z"]]

In [14]:
print(convert_to_array(walking,1))

[2.9719900000000004, 1.5411299999999999, 9.03701]


In [17]:
#Generates arrays of data with a size of window_size and sliding window of size 1
def convert_df_to_array(df, window_size):
  results = []
  start_record = window_size - 1
  #print(start_record, len(df))

  for i in range(start_record, len(df)):
    temp_arr = []
    for r in range(start_record,-1,-1):
      a = convert_to_array(df,i-r)
      temp_arr.append(a)
    results.append(temp_arr)
  return results

In [28]:
results = convert_df_to_array(walking,3)

2 758


In [8]:
len(results)

756

In [30]:
results[1]

[[2.9719900000000004, 1.5411299999999999, 9.03701],
 [2.63141, 1.47678, 9.836889999999999],
 [2.8043299999999998, 1.4272799999999999, 9.983839999999999]]

## Loading Data

In [7]:
content_dir="/content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/*.csv"

In [8]:
files = glob.glob(content_dir)

In [9]:
files

['/content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/jumping-1.csv',
 '/content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/jumping-2.csv',
 '/content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/walking-p1-hand.csv',
 '/content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/walking-p2-hand.csv',
 '/content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/walking-p-hand.csv',
 '/content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/still-p1-hand.csv',
 '/content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/still-p2-hand.csv']

In [1]:
categories_data={}

In [19]:
#Read all the files and segregate them into categories based on the filename, perform a sliding window extraction on the df and put them into a dict 
for f in files:
  basename = os.path.basename(f)
  print(basename)
  category = basename.split("-")[0]

  if category not in categories_data:
    categories_data[category]=[]
    
  df = pd.read_csv(f, delimiter=",",header=None, names=["X","Y","Z"])
  print(category,basename,len(df))
  results = convert_df_to_array(df,window_size)
  print(category,f,len(results))
  categories_data[category] = categories_data[category] + results
  

jumping-1.csv
jumping jumping-1.csv 759
jumping /content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/jumping-1.csv 757
jumping-2.csv
jumping jumping-2.csv 755
jumping /content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/jumping-2.csv 753
walking-p1-hand.csv
walking walking-p1-hand.csv 1293
walking /content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/walking-p1-hand.csv 1291
walking-p2-hand.csv
walking walking-p2-hand.csv 78
walking /content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/walking-p2-hand.csv 76
walking-p-hand.csv
walking walking-p-hand.csv 1325
walking /content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/walking-p-hand.csv 1323
still-p1-hand.csv
still still-p1-hand.csv 1498
still /content/drive/MyDrive/Colab Notebooks/Walking, Standing, Running detection/still-p1-hand.csv 1496
still-p2-hand.csv
still still-p2-hand.csv 488
still /content/drive/MyDrive/Colab Note

In [13]:
categories_data.keys()

dict_keys(['jumping', 'walking', 'still'])

In [41]:
num_categories = categories_data.keys()

In [15]:
len(categories_data["jumping"])

1510

In [40]:
X_all = []
Y_all = None
category_num=0
for i in categories_data.keys():
  
  data = categories_data[i]
  print(i, len(data), category_num)
  X_all =X_all+data

  #Fill Y with the same category number
  Y = np.full((len(data)), category_num)

  if Y_all is None:
    Y_all=Y
  else:
    Y_all = np.concatenate([Y_all, Y])
  category_num=category_num+1

jumping 3020 0
walking 5380 1
still 3964 2


Sanity Check on the data, make sure X_all and Y_all has the same number of rows

In [33]:
print(array(X_all).shape)
print(Y_all.shape)

(12364, 3, 3)
(12364,)


Change Y to categorical since we will be doing prediction on this

In [35]:
Y = tf.keras.utils.to_categorical(
    Y_all, num_classes=len(categories_data.keys()), dtype='float32'
)

In [39]:
Y[3020]

array([0., 1., 0.], dtype=float32)

In [ ]:
X = array(results)

In [ ]:
X.shape

(72, 3, 4)

In [ ]:
X = X.reshape(X.shape[0], X.shape[1], nfeatures)
X.shape

(72, 3, 4)

## Model

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(window_size, nfeatures)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(num_categories,activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X, array(y), epochs=50)

Epoch 1/50
3/3 [==============================] - 2s 24ms/step - loss: 11.4990 - accuracy: 0.5417
Epoch 2/50
3/3 [==============================] - 0s 24ms/step - loss: 13.5601 - accuracy: 0.5694
Epoch 3/50
3/3 [==============================] - 0s 20ms/step - loss: 6.4215 - accuracy: 0.7361
Epoch 4/50
3/3 [==============================] - 0s 29ms/step - loss: 4.0518 - accuracy: 0.6944
Epoch 5/50
3/3 [==============================] - 0s 29ms/step - loss: 2.6981 - accuracy: 0.6389
Epoch 6/50
3/3 [==============================] - 0s 24ms/step - loss: 2.9999 - accuracy: 0.5972
Epoch 7/50
3/3 [==============================] - 0s 29ms/step - loss: 2.9214 - accuracy: 0.5972
Epoch 8/50
3/3 [==============================] - 0s 25ms/step - loss: 1.3963 - accuracy: 0.7083
Epoch 9/50
3/3 [==============================] - 0s 29ms/step - loss: 0.9564 - accuracy: 0.7500
Epoch 10/50
3/3 [==============================] - 0s 20ms/step - loss: 0.9421 - accuracy: 0.8056
Epoch 11/50
3/3 [==========